A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef EQCOMDupireSSSABR < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% SS SABR parameters
        %dynamic
        alpha_0
        alpha_inf
        rho
        nu
        
        %static
        beta
        lambda
        omega
        time_shift
       
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        settleFuturesRef
        atmVols
        
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSSSABR(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                eqCOMDupireSpotGF.alpha_0 = EQModel.modelParams('alpha_0');
                eqCOMDupireSpotGF.alpha_inf = EQModel.modelParams('alpha_inf');
                eqCOMDupireSpotGF.rho = EQModel.modelParams('rho');
                eqCOMDupireSpotGF.nu = EQModel.modelParams('nu');
                
                
                eqCOMDupireSpotGF.beta = EQModel.modelParams('beta');
                eqCOMDupireSpotGF.lambda = EQModel.modelParams('lambda');
                eqCOMDupireSpotGF.omega = EQModel.modelParams('omega');
                eqCOMDupireSpotGF.time_shift = EQModel.modelParams('time_shift');
                
              %% SS SABR model params start
                
              %% SS SABR model params end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                if ~isKey(EQModel.modelParams,'settleFutures')
                    eqCOMDupireSpotGF.settleFutures= [];
                else
                    eqCOMDupireSpotGF.settleFutures= EQModel.modelParams('settleFutures');
                end
                
                if ~isKey(EQModel.modelParams,'settleFuturesRef')
                    eqCOMDupireSpotGF.settleFuturesRef= [];
                else
                    eqCOMDupireSpotGF.settleFuturesRef = EQModel.modelParams('settleFuturesRef');
                end
                
                if ~isKey(EQModel.modelParams,'atmVols')
                    eqCOMDupireSpotGF.atmVols= [];
                else
                    eqCOMDupireSpotGF.atmVols = EQModel.modelParams('atmVols');
                end
                
                if isKey(EQModel.modelParams,'localVolSurface')
                    eqCOMDupireSpotGF.localVolSurface = EQModel.modelParams('localVolSurface');
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = initFuturesStates(eqCOMDupireSpotGF,NMC)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            out =  spotPrice*ones(NMC,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            out = nextFuturesStates;
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end

        function localVol = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry;
            
            %find which strip fromTime belongs to 
            idxS = find(expiry > fromTime);
            if isempty(idxS)
                idxStrip = length(expiry);
            else
                idxStrip = min(idxS);
            end
            
            % find exact timeSlice in the given strip
            localVolTimeStep = eqCOMDupireSpotGF.localVolSurface.localVolTimeStep{idxS};
            localVolSurfaceSABR = eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR{idxS};
            forwardMoneyness = eqCOMDupireSpotGF.localVolSurface.forwardMoneyness{idxS};

            idx = find(localVolTimeStep > fromTime);
            if isempty(idx)
                idxNow = length(localVolTimeStep);
            else
                idxNow = min(idx);
            end
            
            localVolLine = localVolSurfaceSABR(idxNow,:);
%             forwardMoneynessLine = forwardMoneyness(idxNow,:);
            
            localVol =  zeros(length(x),1);
            
            dummyOut = interp1FlatExtrapNew(forwardMoneyness,localVolLine,x','linear');
            dummyOut2 = sqrt(dummyOut);
            localVol = dummyOut2';
            
%             for ii =1:length(x)
%                 dummyVariance = H_interpolation(forwardMoneyness,localVolLine,x(ii),1);
%                 localVol(ii) = sqrt(dummyVariance);
%             end

        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
         function mcmcOut = inductMCForwardNewReducedQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut1 = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            mcmcOut.nextFuturesStates = mcmcOut1.nextX;
            mcmcOut.idxNM = 1;
        end

        function mcmcOut = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
%         spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
%         detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
% 
%          %stochasticFwd(0,T)
%          stateSize = length(modelStates);
%          stochasticFwd = zeros(stateSize,1);
% 
%          for i=1:stateSize
%             stochasticFwd(i) = detFwd * modelStates(i); 
%          end

%          out = stochasticFwd;
         
         out = modelStates;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
%             % piecewise constant interpol using reducedFutures
%             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
%              
%             forwardCurveSize = size(reducedFutures,1);
%             forwardMaturity = reducedFutures(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              fwd_NM = reducedFutures(idxNM,2);
%              
%              
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = computeLocalVolGridSSSABR(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,smileDynamicsAdjFactors)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            %% if we use fixedGrid then we may need to normalize strike using spot otherwise just use 1
%             spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
             
%             spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            strikeMoneynessGridNew = strikeMoneynessGrid;
            
            lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGridNew));
            
            dt = 0.5*0.000001/365.0;
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            

            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd = eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f = eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b = eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGridNew)
                    f_strike_lookup =  lookupMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  lookupMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = lookupMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGridNew(idxkk));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd,strike_lookup,matT);
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
                    f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f,strike_lookup,f_matT);
                    b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b,strike_lookup,b_matT);
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd,f_strike_lookup,matT);
                    b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd,b_strike_lookup,matT);
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeCalendarSpreadOptionMCReducedInt(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = GenerateLocalVolSS(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
            end

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        function out = computeStepdown1SMCReducedInt(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReducedIntEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = SABR_ImpVol(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out = model_vol;
        end
        
        function out = SABR_ImpVol2(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = SABR_ImpVol_Qlib(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            
            dummyQLEpsilon = 2.22045e-16;
            thresHold = dummyQLEpsilon*10.0;
            dummyz2 = abs(z*z);
            if dummyz2 > thresHold
                multiplier = z/Xz;
            else
                multiplier = 1.0 -0.5*rho*z -(3.0*rho*rho-2.0)*z*z/12.0;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = CalibrateToVolSurfaceSSSABR(eqCOMDupireSpotGF,black,calibrationFlag)
%             out = 0.0;
            
            % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
          %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
            
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            % calibration Params
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            params.blackVol = blackVol;
           
            params.fwdMoneyness = fwdMoneyness;
            params.marketStrike = params.fwdMoneyness;
            params.calibrationYN = eqCOMDupireSpotGF.useYN;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            params.lb = eqCOMDupireSpotGF.modelParams('lb');
            params.ub = eqCOMDupireSpotGF.modelParams('ub');
            
            % model params
            params.alpha_0 = eqCOMDupireSpotGF.alpha_0;
            params.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            params.rho = eqCOMDupireSpotGF.rho;
            params.nu = eqCOMDupireSpotGF.nu;
            
            params.beta = eqCOMDupireSpotGF.beta;
            params.lambda = eqCOMDupireSpotGF.lambda;
            params.omega =eqCOMDupireSpotGF.omega;
            params.time_shift =eqCOMDupireSpotGF.time_shift;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.modelParams('calibrationErrorType');
            
            %output 
            out.marketStrikes = params.fwdMoneyness;
            out.fwdMoneyness = params.fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.SSSABROTMPrice = zeros(expirySize,strikeSize);
            out.SSSABRBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global4'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'global4'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global4bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [1e-8,1e-8,-0.9999,1e-8,1e-8,1e-8];
%                     ub = [100.0,100.0,0.9999,100.0,100.0,100.0];
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');

                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'justCalculateError'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                otherwise
                        disp('unimplemented')
            end
            
            eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
            eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
            
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSpotGFSSRSVI < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
        targetStrikes
        
        sviType
        svi_v
        svi_psi
        svi_p
        svi_c
        svi_vTilda
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSRSVI(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SVI params
                if isKey(EQModel.modelParams,'sviType')
                    eqCOMDupireSpotGF.sviType = EQModel.modelParams('sviType');
                    if strcmp(eqCOMDupireSpotGF.sviType,'jumpwing')
                        eqCOMDupireSpotGF.svi_v = EQModel.modelParams('svi_v');
                        eqCOMDupireSpotGF.svi_psi = EQModel.modelParams('svi_psi');
                        eqCOMDupireSpotGF.svi_p = EQModel.modelParams('svi_p');
                        eqCOMDupireSpotGF.svi_c = EQModel.modelParams('svi_c');
                        eqCOMDupireSpotGF.svi_vTilda = EQModel.modelParams('svi_vTilda');
                    end
                else
                    eqCOMDupireSpotGF.sviType = 'jumpwing';
                end
                
              %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
            if idxNow <= numOfCutoffTenor
                lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
                upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
            end
            
            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            
            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            if strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            
            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;
            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
                    out = newOut;
                    
                    % bumped volSurface as output
                    out.shiftedImpVolSurface = outDummy.impVolSurface;
                    out.targetStrikes = targetStrikes;
                    aaa = 1.0;
                    
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSpotGFSSRSVI < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
        targetStrikes
        
        sviType
        svi_v
        svi_psi
        svi_p
        svi_c
        svi_vTilda
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSRSVI(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SVI params
                if isKey(EQModel.modelParams,'sviType')
                    eqCOMDupireSpotGF.sviType = EQModel.modelParams('sviType');
                    if strcmp(eqCOMDupireSpotGF.sviType,'jumpwing')
                        eqCOMDupireSpotGF.svi_v = EQModel.modelParams('svi_v');
                        eqCOMDupireSpotGF.svi_psi = EQModel.modelParams('svi_psi');
                        eqCOMDupireSpotGF.svi_p = EQModel.modelParams('svi_p');
                        eqCOMDupireSpotGF.svi_c = EQModel.modelParams('svi_c');
                        eqCOMDupireSpotGF.svi_vTilda = EQModel.modelParams('svi_vTilda');
                    end
                else
                    eqCOMDupireSpotGF.sviType = 'jumpwing';
                end
                
              %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
            if idxNow <= numOfCutoffTenor
                lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
                upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
            end
            
            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            
            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            if strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            
            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;
            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
                    out = newOut;
                    
                    % bumped volSurface as output
                    out.shiftedImpVolSurface = outDummy.impVolSurface;
                    out.targetStrikes = targetStrikes;
                    aaa = 1.0;
                    
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSpotGFSSRRefSpot < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        % added 20210202
        refSpot
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSRRefSpot(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
                if isKey(EQModel.modelParams,'refSpot')
                    eqCOMDupireSpotGF.refSpot = EQModel.modelParams('refSpot');
                else
                    
                    eqCOMDupireSpotGF.refSpot = EQModel.spot.params('rawData');
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%                 
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','linear');
                localVol = dummyOut';
                
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),3);
%                 end
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','pchip');
                localVol = dummyOut';
                
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotGF(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMC(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % piecewise constant interpol using reducedFutures
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeCalendarSpreadOptionMCReduced(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ_ohVec(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           % added for oveHedgeVector
           overHedgeSpreadVector = stepdownParams.params('overHedgeSpreadVector');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.overHedgeSpreadVector = overHedgeSpreadVector;
           
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpreadVector(i-1)
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpreadVector(i-1));
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaEQGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
%            initX = ones(modelStatesSize,1);
%            nextX = initX;
           
%            comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           comFwd = eqCOMDupireSpotGF.EQSpotGF(currentTime);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               else
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                            payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                            payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                       end
                   else
                       for i=1:modelStatesSize
                            payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                            payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*1.0;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
%                     %apply american option
% %                     comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%                     comFwd = eqCOMDupireSpotGF.EQSpotGF(currentTime);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValueB  = meshB/onePayoffD;
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     if strcmp(vanillaParams.params('callPutFlag'),'C')
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
%                             if payoffStateB.cashflow(j) > contiValueB(j)
%                                 ccc = 1.0;
%                             end
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                         
%                     else
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     end
%                     payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
%                     meshB = payoffStateB.cashflow_npv;
%                     dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
%                     dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
%                     dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaAMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           worstP = zeros(modelStatesSize,length(timeStep));
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(timeStep(1),maturity,nextX);
                worstP(:,1) = comFwd(:);
                    
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
                    worstP(:,idx+1) = comFwd(:);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
           lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
          %% Backward Induction Start (Regression)
            endTimeIdx = length(timeStep)-1;  % valueDate
%             currentTime = timeStep(endTimeIdx);
%             idx= startTimeIdx:1:endTimeIdx
           for i=endTimeIdx:-1:1+1
                % past schedule neglect
                
                
                %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
                comFwd = worstP(:,i);
                currentTime = timeStep(i);
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate

                exerciseValue = zeros(modelStatesSize,1);
                contiValue = zeros(modelStatesSize,1);

                onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);

                % we only use paths that is still alive for regression
                for idx=1:modelStatesSize
                        contiValue(idx) = americanValue.payoff(idx)/onePayoffD;
                end

%                 contiValue = contiValue/10000;

                decisionVar1 = worstP(:,i);
                % we use upto 2nd order polynomial as regression
                % equatioin
                dimensionLS = 4;
                regMatrix = ones(modelStatesSize,dimensionLS);
                for idx_r=1:modelStatesSize
                    regMatrix(idx_r,2) = decisionVar1(idx_r);
                    regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                    regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                end
                % we use only alive path at the execise date as
                % regression input
                beta =zeros(dimensionLS,1);

                numSim = 0;
                regMatrix2 = zeros(dimensionLS,dimensionLS);
                contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                aliveVector = zeros(modelStatesSize,1);
                for idx=1:modelStatesSize
                    if worstP(idx,i) <= strike % ITM path at exercise date
                        numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                        aliveVector(idx) = 1;
                        exerciseValue(idx) = max(-worstP(idx,i) + strike,0.0);
                        
                    else % for OTM path we do nothing
                        exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                    end

                end

                % if we have ITM paths, then we do the regression
                if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
                    for ii=1:dimensionLS
                        regVi = regMatrix(:,ii);
                        for jj=1:dimensionLS
                            regVj = regMatrix(:,jj);
                            for idx3=1:modelStatesSize
                                regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                            end
                        end
                        for idx3=1:modelStatesSize
                            contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                        end
                    end

                    beta = regress(contiValue2,regMatrix2);
                    regContiValue = regMatrix*beta;

                    for idx=1:modelStatesSize
                        if worstP(idx,i) <= strike %live path
                            % regContiValue is only used for decision making
                            if regContiValue(idx) <= exerciseValue(idx) 
                                contiValue(idx) = exerciseValue(idx);
                            end
                        else  % ITM path we do nothing
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                else % there is no alive path then we do nothing
                    for idx=1:modelStatesSize
                        contiValue(idx) = exerciseValue(idx);
                    end
                end

                % discount back
                for idx=1:modelStatesSize
                    americanValue.payoff(idx) = contiValue(idx)*onePayoffD;
                end

%                 unhitValue.payoff = unhitValue.payoff * 10000;


               
                % we are at the start of the period
                
            end
            
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolSimple(eqCOMDupireSpotGF,targetExpiry,targetStrikes)

            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes;
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpirySimple(targetExpiry(i),strikePerExpiry);
                targetImpVolPerExpiry = outDummy.impVol;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            errorTryTrail = zeros(maxIter,1);
            newLocalVolTrail = zeros(maxIter,optParams.strikeSize);
            modelImpVolTrail = zeros(maxIter,optParams.strikeSize);
            
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    
                    if n== 47
                        aaa = 1.0;
                    end
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    errorTryTrail(n+1) = errorTry;
                    newLocalVolTrail(n+1,:) = out.newLocalVol;
                    modelImpVolTrail(n+1,:) = out.modelImpVol;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                  %% compute interpolated implied surface for targetStrikes(forwardmoneyness);
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    
                    dummyStrikeFwd = 0.5:0.1:1.5;
                    targetStrikesFwd = zeros(length(targetExpiry),length(dummyStrikeFwd));
                    
                    
                 %% Error Correction 20210202 from commit 342 20200402
                   
                 %% before correction
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         for j=1:length(dummyStrikeFwd)
%                             targetStrikesFwd(i,j) = dummyStrikeFwd(j);
% %                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
% %                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
% %                             for j=1:size(targetStrikes,2)
% %                                 targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
% %                             end
%                         end
%                     end
                    
                 %% after correction for fwdMoneyness
                 if size(out.fwdMoneyness,1) == 1
                        slope = eqCOMDupireSpotGF.slope;
                        nmFwd = eqCOMDupireSpotGF.spot.params('rawData');
                        referenceNmFwd = eqCOMDupireSpotGF.refSpot;
                        for i=1:length(targetExpiry)
                            for j=1:length(dummyStrikeFwd)
        %                             targetStrikesFwd(i,j) = dummyStrikeFwd(j);
        %                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
        %                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                                targetStrikesFwd(i,j) = dummyStrikeFwd(j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                            end
                        end
                 else  
                    %% after correction for spotdMoneyness
                        slope = eqCOMDupireSpotGF.slope;
                        nmFwd = eqCOMDupireSpotGF.spot.params('rawData');
                        referenceNmFwd = eqCOMDupireSpotGF.refSpot;
                        for i=1:length(targetExpiry)
                            for j=1:length(dummyStrikeFwd)
        %                             targetStrikesFwd(i,j) = dummyStrikeFwd(j);
        %                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
        %                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                                targetStrikesFwd(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                            end
                        end
                 end
                 %% correction ended
                   
                    eqCOMDupireSpotGF.targetStrikes = targetStrikesFwd;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikesFwd);
                    out.forwardImpVol = outDummy.impVolSurface;
                    
                    for i=1:length(targetExpiry)
                        for j=1:size(targetStrikesFwd,2)
                            out.totalVar(i,j) = out.forwardImpVol(i,j)*out.forwardImpVol(i,j)*targetExpiry(i)/365.0;
                        end
                    end
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
%                     %% compute interpolated implied surface for targetStrikes(fixedStrikes);
%                     dummyStrikeFG = 15:5:65;
%                     targetStrikeFG = zeros(length(targetExpiry),length(dummyStrikeFG));
%                     
%                     for i=1:length(targetExpiry)
%                         for j=1:length(targetStrikeFG)
%                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                             targetStrikeFG(i,j) = dummyStrikeFG(j)/nmFwd;
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikeFG;
%                     
%                     outDummy2 = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikeFG);
%                     out.forwardImpVol2 = outDummy2.impVolSurface;
%                     out.cValueSurface2 = outDummy2.cValueSurface;
%                     out.pValueSurface2 = outDummy2.pValueSurface;
%                     out.probMeshInt2 = outDummy2.probMeshInt;
%                     
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSpotGFSSRRefSpot < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        % added 20210202
        refSpot
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSRRefSpot(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
                if isKey(EQModel.modelParams,'refSpot')
                    eqCOMDupireSpotGF.refSpot = EQModel.modelParams('refSpot');
                else
                    
                    eqCOMDupireSpotGF.refSpot = EQModel.spot.params('rawData');
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%                 
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','linear');
                localVol = dummyOut';
                
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),3);
%                 end
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','pchip');
                localVol = dummyOut';
                
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotGF(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMC(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % piecewise constant interpol using reducedFutures
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeCalendarSpreadOptionMCReduced(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ_ohVec(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           % added for oveHedgeVector
           overHedgeSpreadVector = stepdownParams.params('overHedgeSpreadVector');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.overHedgeSpreadVector = overHedgeSpreadVector;
           
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpreadVector(i-1)
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpreadVector(i-1));
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaEQGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
%            initX = ones(modelStatesSize,1);
%            nextX = initX;
           
%            comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           comFwd = eqCOMDupireSpotGF.EQSpotGF(currentTime);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               else
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                            payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                            payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                       end
                   else
                       for i=1:modelStatesSize
                            payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                            payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*1.0;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
%                     %apply american option
% %                     comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%                     comFwd = eqCOMDupireSpotGF.EQSpotGF(currentTime);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValueB  = meshB/onePayoffD;
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     if strcmp(vanillaParams.params('callPutFlag'),'C')
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
%                             if payoffStateB.cashflow(j) > contiValueB(j)
%                                 ccc = 1.0;
%                             end
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                         
%                     else
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     end
%                     payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
%                     meshB = payoffStateB.cashflow_npv;
%                     dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
%                     dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
%                     dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaAMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           worstP = zeros(modelStatesSize,length(timeStep));
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(timeStep(1),maturity,nextX);
                worstP(:,1) = comFwd(:);
                    
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
                    worstP(:,idx+1) = comFwd(:);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
           lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
          %% Backward Induction Start (Regression)
            endTimeIdx = length(timeStep)-1;  % valueDate
%             currentTime = timeStep(endTimeIdx);
%             idx= startTimeIdx:1:endTimeIdx
           for i=endTimeIdx:-1:1+1
                % past schedule neglect
                
                
                %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
                comFwd = worstP(:,i);
                currentTime = timeStep(i);
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate

                exerciseValue = zeros(modelStatesSize,1);
                contiValue = zeros(modelStatesSize,1);

                onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);

                % we only use paths that is still alive for regression
                for idx=1:modelStatesSize
                        contiValue(idx) = americanValue.payoff(idx)/onePayoffD;
                end

%                 contiValue = contiValue/10000;

                decisionVar1 = worstP(:,i);
                % we use upto 2nd order polynomial as regression
                % equatioin
                dimensionLS = 4;
                regMatrix = ones(modelStatesSize,dimensionLS);
                for idx_r=1:modelStatesSize
                    regMatrix(idx_r,2) = decisionVar1(idx_r);
                    regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                    regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                end
                % we use only alive path at the execise date as
                % regression input
                beta =zeros(dimensionLS,1);

                numSim = 0;
                regMatrix2 = zeros(dimensionLS,dimensionLS);
                contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                aliveVector = zeros(modelStatesSize,1);
                for idx=1:modelStatesSize
                    if worstP(idx,i) <= strike % ITM path at exercise date
                        numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                        aliveVector(idx) = 1;
                        exerciseValue(idx) = max(-worstP(idx,i) + strike,0.0);
                        
                    else % for OTM path we do nothing
                        exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                    end

                end

                % if we have ITM paths, then we do the regression
                if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
                    for ii=1:dimensionLS
                        regVi = regMatrix(:,ii);
                        for jj=1:dimensionLS
                            regVj = regMatrix(:,jj);
                            for idx3=1:modelStatesSize
                                regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                            end
                        end
                        for idx3=1:modelStatesSize
                            contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                        end
                    end

                    beta = regress(contiValue2,regMatrix2);
                    regContiValue = regMatrix*beta;

                    for idx=1:modelStatesSize
                        if worstP(idx,i) <= strike %live path
                            % regContiValue is only used for decision making
                            if regContiValue(idx) <= exerciseValue(idx) 
                                contiValue(idx) = exerciseValue(idx);
                            end
                        else  % ITM path we do nothing
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                else % there is no alive path then we do nothing
                    for idx=1:modelStatesSize
                        contiValue(idx) = exerciseValue(idx);
                    end
                end

                % discount back
                for idx=1:modelStatesSize
                    americanValue.payoff(idx) = contiValue(idx)*onePayoffD;
                end

%                 unhitValue.payoff = unhitValue.payoff * 10000;


               
                % we are at the start of the period
                
            end
            
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolSimple(eqCOMDupireSpotGF,targetExpiry,targetStrikes)

            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes;
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpirySimple(targetExpiry(i),strikePerExpiry);
                targetImpVolPerExpiry = outDummy.impVol;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            errorTryTrail = zeros(maxIter,1);
            newLocalVolTrail = zeros(maxIter,optParams.strikeSize);
            modelImpVolTrail = zeros(maxIter,optParams.strikeSize);
            
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    
                    if n== 47
                        aaa = 1.0;
                    end
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    errorTryTrail(n+1) = errorTry;
                    newLocalVolTrail(n+1,:) = out.newLocalVol;
                    modelImpVolTrail(n+1,:) = out.modelImpVol;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                  %% compute interpolated implied surface for targetStrikes(forwardmoneyness);
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    
                    dummyStrikeFwd = 0.5:0.1:1.5;
                    targetStrikesFwd = zeros(length(targetExpiry),length(dummyStrikeFwd));
                    
                    
                 %% Error Correction 20210202 from commit 342 20200402
                   
                 %% before correction
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         for j=1:length(dummyStrikeFwd)
%                             targetStrikesFwd(i,j) = dummyStrikeFwd(j);
% %                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
% %                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
% %                             for j=1:size(targetStrikes,2)
% %                                 targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
% %                             end
%                         end
%                     end
                    
                 %% after correction for fwdMoneyness
                 if size(out.fwdMoneyness,1) == 1
                        slope = eqCOMDupireSpotGF.slope;
                        nmFwd = eqCOMDupireSpotGF.spot.params('rawData');
                        referenceNmFwd = eqCOMDupireSpotGF.refSpot;
                        for i=1:length(targetExpiry)
                            for j=1:length(dummyStrikeFwd)
        %                             targetStrikesFwd(i,j) = dummyStrikeFwd(j);
        %                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
        %                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                                targetStrikesFwd(i,j) = dummyStrikeFwd(j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                            end
                        end
                 else  
                    %% after correction for spotdMoneyness
                        slope = eqCOMDupireSpotGF.slope;
                        nmFwd = eqCOMDupireSpotGF.spot.params('rawData');
                        referenceNmFwd = eqCOMDupireSpotGF.refSpot;
                        for i=1:length(targetExpiry)
                            for j=1:length(dummyStrikeFwd)
        %                             targetStrikesFwd(i,j) = dummyStrikeFwd(j);
        %                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
        %                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                                targetStrikesFwd(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                            end
                        end
                 end
                 %% correction ended
                   
                    eqCOMDupireSpotGF.targetStrikes = targetStrikesFwd;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikesFwd);
                    out.forwardImpVol = outDummy.impVolSurface;
                    
                    for i=1:length(targetExpiry)
                        for j=1:size(targetStrikesFwd,2)
                            out.totalVar(i,j) = out.forwardImpVol(i,j)*out.forwardImpVol(i,j)*targetExpiry(i)/365.0;
                        end
                    end
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
%                     %% compute interpolated implied surface for targetStrikes(fixedStrikes);
%                     dummyStrikeFG = 15:5:65;
%                     targetStrikeFG = zeros(length(targetExpiry),length(dummyStrikeFG));
%                     
%                     for i=1:length(targetExpiry)
%                         for j=1:length(targetStrikeFG)
%                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                             targetStrikeFG(i,j) = dummyStrikeFG(j)/nmFwd;
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikeFG;
%                     
%                     outDummy2 = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikeFG);
%                     out.forwardImpVol2 = outDummy2.impVolSurface;
%                     out.cValueSurface2 = outDummy2.cValueSurface;
%                     out.pValueSurface2 = outDummy2.pValueSurface;
%                     out.probMeshInt2 = outDummy2.probMeshInt;
%                     
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
%classdef EQCOMDupireSpotGFSSR < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSR(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%                 
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','linear');
                localVol = dummyOut';
                
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),3);
%                 end
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','pchip');
                localVol = dummyOut';
                
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMC(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % piecewise constant interpol using reducedFutures
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeCalendarSpreadOptionMCReduced(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaAMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           worstP = zeros(modelStatesSize,length(timeStep));
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(timeStep(1),maturity,nextX);
                worstP(:,1) = comFwd(:);
                    
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
                    worstP(:,idx+1) = comFwd(:);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
           lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
          %% Backward Induction Start (Regression)
            endTimeIdx = length(timeStep)-1;  % valueDate
%             currentTime = timeStep(endTimeIdx);
%             idx= startTimeIdx:1:endTimeIdx
           for i=endTimeIdx:-1:1+1
                % past schedule neglect
                
                
                %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
                comFwd = worstP(:,i);
                currentTime = timeStep(i);
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate

                exerciseValue = zeros(modelStatesSize,1);
                contiValue = zeros(modelStatesSize,1);

                onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);

                % we only use paths that is still alive for regression
                for idx=1:modelStatesSize
                        contiValue(idx) = americanValue.payoff(idx)/onePayoffD;
                end

%                 contiValue = contiValue/10000;

                decisionVar1 = worstP(:,i);
                % we use upto 2nd order polynomial as regression
                % equatioin
                dimensionLS = 4;
                regMatrix = ones(modelStatesSize,dimensionLS);
                for idx_r=1:modelStatesSize
                    regMatrix(idx_r,2) = decisionVar1(idx_r);
                    regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                    regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                end
                % we use only alive path at the execise date as
                % regression input
                beta =zeros(dimensionLS,1);

                numSim = 0;
                regMatrix2 = zeros(dimensionLS,dimensionLS);
                contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                aliveVector = zeros(modelStatesSize,1);
                for idx=1:modelStatesSize
                    if worstP(idx,i) <= strike % ITM path at exercise date
                        numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                        aliveVector(idx) = 1;
                        exerciseValue(idx) = max(-worstP(idx,i) + strike,0.0);
                        
                    else % for OTM path we do nothing
                        exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                    end

                end

                % if we have ITM paths, then we do the regression
                if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
                    for ii=1:dimensionLS
                        regVi = regMatrix(:,ii);
                        for jj=1:dimensionLS
                            regVj = regMatrix(:,jj);
                            for idx3=1:modelStatesSize
                                regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                            end
                        end
                        for idx3=1:modelStatesSize
                            contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                        end
                    end

                    beta = regress(contiValue2,regMatrix2);
                    regContiValue = regMatrix*beta;

                    for idx=1:modelStatesSize
                        if worstP(idx,i) <= strike %live path
                            % regContiValue is only used for decision making
                            if regContiValue(idx) <= exerciseValue(idx) 
                                contiValue(idx) = exerciseValue(idx);
                            end
                        else  % ITM path we do nothing
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                else % there is no alive path then we do nothing
                    for idx=1:modelStatesSize
                        contiValue(idx) = exerciseValue(idx);
                    end
                end

                % discount back
                for idx=1:modelStatesSize
                    americanValue.payoff(idx) = contiValue(idx)*onePayoffD;
                end

%                 unhitValue.payoff = unhitValue.payoff * 10000;


               
                % we are at the start of the period
                
            end
            
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolSimple(eqCOMDupireSpotGF,targetExpiry,targetStrikes)

            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes;
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpirySimple(targetExpiry(i),strikePerExpiry);
                targetImpVolPerExpiry = outDummy.impVol;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                  %% compute interpolated implied surface for targetStrikes(forwardmoneyness);
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    
                    dummyStrikeFwd = 0.5:0.1:1.5;
                    targetStrikesFwd = zeros(length(targetExpiry),length(dummyStrikeFwd));
                    
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        for j=1:length(dummyStrikeFwd)
                            targetStrikesFwd(i,j) = dummyStrikeFwd(j);
%                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                             for j=1:size(targetStrikes,2)
%                                 targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                             end
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikesFwd;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikesFwd);
                    out.forwardImpVol = outDummy.impVolSurface;
                    
                    for i=1:length(targetExpiry)
                        for j=1:size(targetStrikesFwd,2)
                            out.totalVar(i,j) = out.forwardImpVol(i,j)*out.forwardImpVol(i,j)*targetExpiry(i)/365.0;
                        end
                    end
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
%                     %% compute interpolated implied surface for targetStrikes(fixedStrikes);
%                     dummyStrikeFG = 15:5:65;
%                     targetStrikeFG = zeros(length(targetExpiry),length(dummyStrikeFG));
%                     
%                     for i=1:length(targetExpiry)
%                         for j=1:length(targetStrikeFG)
%                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                             targetStrikeFG(i,j) = dummyStrikeFG(j)/nmFwd;
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikeFG;
%                     
%                     outDummy2 = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikeFG);
%                     out.forwardImpVol2 = outDummy2.impVolSurface;
%                     out.cValueSurface2 = outDummy2.cValueSurface;
%                     out.pValueSurface2 = outDummy2.pValueSurface;
%                     out.probMeshInt2 = outDummy2.probMeshInt;
                    
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end


classdef EQCOMDupireSpotGFSSR < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSR(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%                 
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','linear');
                localVol = dummyOut';
                
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),3);
%                 end
                dummyOut = interp1FlatExtrapNew(fwdMoneynessPerExpiry,volProxy,x','pchip');
                localVol = dummyOut';
                
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMC(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % piecewise constant interpol using reducedFutures
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeCalendarSpreadOptionMCReduced(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaAMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           worstP = zeros(modelStatesSize,length(timeStep));
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(timeStep(1),maturity,nextX);
                worstP(:,1) = comFwd(:);
                    
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
                    worstP(:,idx+1) = comFwd(:);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
           lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
          %% Backward Induction Start (Regression)
            endTimeIdx = length(timeStep)-1;  % valueDate
%             currentTime = timeStep(endTimeIdx);
%             idx= startTimeIdx:1:endTimeIdx
           for i=endTimeIdx:-1:1+1
                % past schedule neglect
                
                
                %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
                comFwd = worstP(:,i);
                currentTime = timeStep(i);
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate

                exerciseValue = zeros(modelStatesSize,1);
                contiValue = zeros(modelStatesSize,1);

                onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);

                % we only use paths that is still alive for regression
                for idx=1:modelStatesSize
                        contiValue(idx) = americanValue.payoff(idx)/onePayoffD;
                end

%                 contiValue = contiValue/10000;

                decisionVar1 = worstP(:,i);
                % we use upto 2nd order polynomial as regression
                % equatioin
                dimensionLS = 4;
                regMatrix = ones(modelStatesSize,dimensionLS);
                for idx_r=1:modelStatesSize
                    regMatrix(idx_r,2) = decisionVar1(idx_r);
                    regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                    regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                end
                % we use only alive path at the execise date as
                % regression input
                beta =zeros(dimensionLS,1);

                numSim = 0;
                regMatrix2 = zeros(dimensionLS,dimensionLS);
                contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                aliveVector = zeros(modelStatesSize,1);
                for idx=1:modelStatesSize
                    if worstP(idx,i) <= strike % ITM path at exercise date
                        numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                        aliveVector(idx) = 1;
                        exerciseValue(idx) = max(-worstP(idx,i) + strike,0.0);
                        
                    else % for OTM path we do nothing
                        exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                    end

                end

                % if we have ITM paths, then we do the regression
                if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
                    for ii=1:dimensionLS
                        regVi = regMatrix(:,ii);
                        for jj=1:dimensionLS
                            regVj = regMatrix(:,jj);
                            for idx3=1:modelStatesSize
                                regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                            end
                        end
                        for idx3=1:modelStatesSize
                            contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                        end
                    end

                    beta = regress(contiValue2,regMatrix2);
                    regContiValue = regMatrix*beta;

                    for idx=1:modelStatesSize
                        if worstP(idx,i) <= strike %live path
                            % regContiValue is only used for decision making
                            if regContiValue(idx) <= exerciseValue(idx) 
                                contiValue(idx) = exerciseValue(idx);
                            end
                        else  % ITM path we do nothing
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                else % there is no alive path then we do nothing
                    for idx=1:modelStatesSize
                        contiValue(idx) = exerciseValue(idx);
                    end
                end

                % discount back
                for idx=1:modelStatesSize
                    americanValue.payoff(idx) = contiValue(idx)*onePayoffD;
                end

%                 unhitValue.payoff = unhitValue.payoff * 10000;


               
                % we are at the start of the period
                
            end
            
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolSimple(eqCOMDupireSpotGF,targetExpiry,targetStrikes)

            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes;
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpirySimple(targetExpiry(i),strikePerExpiry);
                targetImpVolPerExpiry = outDummy.impVol;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                  %% compute interpolated implied surface for targetStrikes(forwardmoneyness);
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    
                    dummyStrikeFwd = 0.5:0.1:1.5;
                    targetStrikesFwd = zeros(length(targetExpiry),length(dummyStrikeFwd));
                    
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        for j=1:length(dummyStrikeFwd)
                            targetStrikesFwd(i,j) = dummyStrikeFwd(j);
%                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                             referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                             for j=1:size(targetStrikes,2)
%                                 targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                             end
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikesFwd;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikesFwd);
                    out.forwardImpVol = outDummy.impVolSurface;
                    
                    for i=1:length(targetExpiry)
                        for j=1:size(targetStrikesFwd,2)
                            out.totalVar(i,j) = out.forwardImpVol(i,j)*out.forwardImpVol(i,j)*targetExpiry(i)/365.0;
                        end
                    end
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
%                     %% compute interpolated implied surface for targetStrikes(fixedStrikes);
%                     dummyStrikeFG = 15:5:65;
%                     targetStrikeFG = zeros(length(targetExpiry),length(dummyStrikeFG));
%                     
%                     for i=1:length(targetExpiry)
%                         for j=1:length(targetStrikeFG)
%                             nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                             targetStrikeFG(i,j) = dummyStrikeFG(j)/nmFwd;
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikeFG;
%                     
%                     outDummy2 = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikeFG);
%                     out.forwardImpVol2 = outDummy2.impVolSurface;
%                     out.cValueSurface2 = outDummy2.cValueSurface;
%                     out.pValueSurface2 = outDummy2.pValueSurface;
%                     out.probMeshInt2 = outDummy2.probMeshInt;
                    
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSpotGFSSRNU < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        deltaH
        useYN
        deltaCutOff

        %nonUniform
        Ks
        D2    
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSRNU(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                
                %NU
                eqCOMDupireSpotGF.Ks = eqCOMDupireSpotGF.modelParams('Ks');
                eqCOMDupireSpotGF.D2 = eqCOMDupireSpotGF.modelParams('D2');
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);

            
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                A_rg = full(L);
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            else
                error('not implemented');
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end

        function out = Solve1DGFNU(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
             
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*Ks(i)*Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;
            
            dT = (fromTime - toTime)/365.0;
            
            A_rg = full(L);
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*Ks(i)*Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;
            A_rg = full(L);
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,maturity);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            
            size = length(lastP);
            %filling end
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*Ks(i)*Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;

            dT = dTDays/365.0;
            A_rg = full(L);
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                idxhh = find(optParams.marketStrikes(i) >= optParams.Ks);
                if isempty(idxhh)
                    error('marketStrikes is not in the grid!')
                end
                optParams.ipos(i) = max(idxhh);
%                 optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                useYN = ones(expirySize,strikeSize);
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
                % fixedStrike Vol is the same as slope 1.0 in the smile dynamics    
%                 eqCOMDupireSpotGF.slope = 1.0;
                for k=1:expirySize
                     fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                        dummyD1 = -log(fwdMoneyness(k,j))/dummyStd + 0.5*dummyStd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            deltaH(k,j) = H_ncdf(dummyD1*-1)*-1;
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            deltaH(k,j) = H_ncdf(dummyD1);
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);

                        if abs(deltaH(k,j)) < eqCOMDupireSpotGF.deltaCutOff
                            useYN(k,j) = 0;
                        else
                            useYN(k,j) = 1;
                        end
                    end
                end
                
                eqCOMDupireSpotGF.useYN = useYN;
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
%             params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            params.Pricingidx = eqCOMDupireSpotGF.modelParams('Pricingidx');
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    out = newOut;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSpotGFSSRNU < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        deltaH
        useYN
        deltaCutOff

        %nonUniform
        Ks
        D2    
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSpotGFSSRNU(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                
                %NU
                eqCOMDupireSpotGF.Ks = eqCOMDupireSpotGF.modelParams('Ks');
                eqCOMDupireSpotGF.D2 = eqCOMDupireSpotGF.modelParams('D2');
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);

            
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                A_rg = full(L);
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            else
                error('not implemented');
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end

        function out = Solve1DGFNU(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
             
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*Ks(i)*Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;
            
            dT = (fromTime - toTime)/365.0;
            
            A_rg = full(L);
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*Ks(i)*Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;
            A_rg = full(L);
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
        spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
        detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

         %stochasticFwd(0,T)
         stateSize = length(modelStates);
         stochasticFwd = zeros(stateSize,1);

         for i=1:stateSize
            stochasticFwd(i) = detFwd * modelStates(i); 
         end

         out = stochasticFwd;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,maturity);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            
            size = length(lastP);
            %filling end
            diffusionCoeff = zeros(size,1); 
            for i=1:size
                diffusionCoeff(i) = 0.5*proxy(i)*proxy(i)*Ks(i)*Ks(i);
            end

            L = sparse(2:size-1,2:size-1,diffusionCoeff(2:end-1),size,size)*eqCOMDupireSpotGF.D2;

            dT = dTDays/365.0;
            A_rg = full(L);
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                idxhh = find(optParams.marketStrikes(i) >= optParams.Ks);
                if isempty(idxhh)
                    error('marketStrikes is not in the grid!')
                end
                optParams.ipos(i) = max(idxhh);
%                 optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                useYN = ones(expirySize,strikeSize);
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
                % fixedStrike Vol is the same as slope 1.0 in the smile dynamics    
%                 eqCOMDupireSpotGF.slope = 1.0;
                for k=1:expirySize
                     fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                        dummyD1 = -log(fwdMoneyness(k,j))/dummyStd + 0.5*dummyStd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            deltaH(k,j) = H_ncdf(dummyD1*-1)*-1;
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            deltaH(k,j) = H_ncdf(dummyD1);
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);

                        if abs(deltaH(k,j)) < eqCOMDupireSpotGF.deltaCutOff
                            useYN(k,j) = 0;
                        else
                            useYN(k,j) = 1;
                        end
                    end
                end
                
                eqCOMDupireSpotGF.useYN = useYN;
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
%             params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            params.Pricingidx = eqCOMDupireSpotGF.modelParams('Pricingidx');
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    out = newOut;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
